In [1]:
import os
import nltk
import string
import re
import numpy as np

parser_dir = "incremental-top-down-parser"

save_dir = "features"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [2]:
def get_surprisal(row):
    all_num = re.findall(r"[-+]?\d*\.\d+|\d+", row)
    print(all_num)
    return float(all_num[-7])

In [3]:
hp_text = open("chapter9.txt","r").readlines()
all_text = []
para = ""
for i in range(len(hp_text)):
    hp_text[i] = hp_text[i].strip()
    if hp_text[i] == '"...':
        hp_text[i] = '"' # string causes a seg fault in the parser
    if hp_text[i][0] == '(':
        hp_text[i] = "," + hp_text[i][1:] # ( causes a seg fault in the parser
    if ')' in hp_text[i]:
        hp_text[i] = hp_text[i].replace(")","") # ) causes a seg fault in the parser
    if hp_text[i]!="+":
        para += hp_text[i] + " "
    else:
        all_text.append(para)
        para = ""

In [4]:
ptr = 0
all_surprisal = []
for i in range(len(hp_text)):
    all_surprisal.append([])

len_remaining = len(hp_text[ptr])
for para in all_text:
    all_sents = nltk.sent_tokenize(para)
    for sent_number in range(len(all_sents)):
        sent = all_sents[sent_number]
        if sent == '..."': # useless sentence, needs to be skipped
            print('Encountered ..." skipping it')
            ptr += 1
            while(ptr < len(hp_text) and hp_text[ptr] == "+"):
                ptr += 1
            len_remaining = len(hp_text[ptr])
            continue
        sent = sent.translate(str.maketrans({key: " {0} ".format(key) for key in string.punctuation})).strip()
        inp_f = open("input.txt", "w+")
        inp_f.write(sent)
        inp_f.close()
        os.system("{}/bin/tdparse -p -F output.txt {}/parse.model.wsj.slc.p05 input.txt".format(parser_dir, parser_dir))
        
        output = open("output.txt", "r").readlines()
        print(sent)
        
        tokens_ori = sent.split(" ")
        tokens = []
        for j in range(len(tokens_ori)): # remove whitespace tokens
            if len(tokens_ori[j].strip()) != 0:
                tokens.append(tokens_ori[j])
        
        k = 0
        for j in range(len(tokens)):
            len_remaining -= len(tokens[j])
            if len_remaining == 0:
                if hp_text[ptr] == '"':
                    ptr += 1
                    len_remaining = len(hp_text[ptr])
                    continue
                print("Word completed by {} len remaining = {}".format(tokens[j], len_remaining))
                
                row = output[k+1]
                surprisal = get_surprisal(row)
                all_surprisal[ptr].append(surprisal)
                
                k += 1
                
                print("Word = {}".format(hp_text[ptr]))
                print("Row = {}".format(row))
                print("All surprisal = {}".format(all_surprisal[ptr]))
                
                ptr += 1
                while(ptr < len(hp_text) and hp_text[ptr] == "+"):
                    ptr += 1
                if ptr < len(hp_text):
                    len_remaining = len(hp_text[ptr])
            else:
                print("Skipped {} len remaining = {}".format(tokens[j], len_remaining))
                if tokens[j] != '"' and ((j == len(tokens) - 1) or (tokens[j+1] != '"')):
                    row = output[k+1]
                    surprisal = get_surprisal(row)
                    all_surprisal[ptr].append(surprisal)
                    k += 1
        print()

Harry had never believed he would meet a boy he hated more than Dudley ,  but that was before he met Draco Malfoy .
Word completed by Harry len remaining = 0
['1', '8.984', '8.984', '1.600', '7.384', '0.329', '1.90', '1.00', '0.90', '3.9']
Word = Harry
Row = pfix:1          Harry   8.984  8.984 1.600  7.384  0.329  1.90  1.00  0.90  3.9

All surprisal = [1.6]
Word completed by had len remaining = 0
['2', '16.008', '7.024', '4.133', '2.891', '0.011', '2.00', '1.11', '1.11', '3.0']
Word = had
Row = pfix:2            had  16.008  7.024 4.133  2.891  0.011  2.00  1.11  1.11  3.0

All surprisal = [4.133]
Word completed by never len remaining = 0
['3', '22.793', '6.785', '2.707', '4.078', '0.606', '3.26', '1.00', '0.74', '2.3']
Word = never
Row = pfix:3          never  22.793  6.785 2.707  4.078  0.606  3.26  1.00  0.74  2.3

All surprisal = [2.707]
Word completed by believed len remaining = 0
['4', '29.972', '7.179', '0.880', '6.298', '0.752', '3.00', '0.82', '0.82', '2.6']
Word = believed


Or at least ,  they didn ' t until they spotted a notice pinned up in the Gryffindor common room that made them all groan .
Word completed by Or len remaining = 0
['1', '7.213', '7.213', '2.823', '4.390', '1.104', '1.02', '1.00', '0.58', '3.0']
Word = Or
Row = pfix:1             Or   7.213  7.213 2.823  4.390  1.104  1.02  1.00  0.58  3.0

All surprisal = [2.823]
Word completed by at len remaining = 0
['2', '12.275', '5.063', '2.471', '2.591', '1.078', '2.04', '1.48', '1.18', '2.0']
Word = at
Row = pfix:2             at  12.275  5.063 2.471  2.591  1.078  2.04  1.48  1.18  2.0

All surprisal = [2.471]
Skipped least len remaining = 1
['3', '15.228', '2.953', '2.855', '0.098', '1.053', '2.14', '0.96', '0.95', '1.0']
Word completed by , len remaining = 0
['4', '17.335', '2.107', '2.107', '0.000', '0.994', '1.05', '1.05', '1.04', '2.1']
Word = least,
Row = pfix:4              ,  17.335  2.107 2.107  0.000  0.994  1.05  1.05  1.04  2.1

All surprisal = [2.855, 2.107]
Word completed by they 

" Typical ,  "  said Harry darkly .
Skipped " len remaining = 9
Skipped Typical len remaining = 2
['1', '8.318', '8.318', '3.154', '5.163', '0.539', '1.14', '1.00', '0.86', '3.1']
Skipped , len remaining = 1
Word completed by " len remaining = 0
['2', '11.636', '3.318', '3.318', '0.000', '1.463', '1.28', '0.38', '0.00', '1.9']
Word = "Typical,"
Row = pfix:2              ,  11.636  3.318 3.318  0.000  1.463  1.28  0.38  0.00  1.9

All surprisal = [3.154, 3.318]
Word completed by said len remaining = 0
['3', '16.907', '5.271', '3.279', '1.993', '0.523', '2.10', '2.66', '2.55', '2.1']
Word = said
Row = pfix:3           said  16.907  5.271 3.279  1.993  0.523  2.10  2.66  2.55  2.1

All surprisal = [3.279]
Word completed by Harry len remaining = 0
['4', '26.448', '9.541', '1.265', '8.276', '0.433', '2.18', '1.05', '1.03', '3.0']
Word = Harry
Row = pfix:4          Harry  26.448  9.541 1.265  8.276  0.433  2.18  1.05  1.03  3.0

All surprisal = [1.265]
Skipped darkly len remaining = 1
['5', 

" You don ' t know that you ' ll make a fool of yourself ,  "  said Ron reasonably .
Skipped " len remaining = 3
Word completed by You len remaining = 0
['1', '5.756', '5.756', '2.389', '3.366', '0.007', '2.00', '1.00', '1.00', '4.0']
Word = "You
Row = pfix:1            You   5.756  5.756 2.389  3.366  0.007  2.00  1.00  1.00  4.0

All surprisal = [2.389]
Skipped don len remaining = 2
['2', '22.584', '16.829', '6.011', '10.818', '0.001', '2.00', '1.00', '1.00', '3.0']
Skipped ' len remaining = 1
['3', '34.270', '11.686', '7.892', '3.794', '0.931', '3.15', '0.99', '0.64', '2.4']
Word completed by t len remaining = 0
['4', '45.304', '11.034', '1.747', '9.286', '2.643', '4.33', '0.88', '0.39', '2.4']
Word = don't
Row = pfix:4              t  45.304 11.034 1.747  9.286  2.643  4.33  0.88  0.39  2.4

All surprisal = [6.011, 7.892, 1.747]
Word completed by know len remaining = 0
['5', '53.902', '8.597', '2.760', '5.838', '1.647', '4.24', '2.37', '2.06', '3.5']
Word = know
Row = pfix:5       

Malfoy certainly did talk about flying a lot .
Word completed by Malfoy len remaining = 0
['1', '5.847', '5.847', '1.062', '4.785', '1.560', '1.92', '1.00', '0.54', '3.9']
Word = Malfoy
Row = pfix:1         Malfoy   5.847  5.847 1.062  4.785  1.560  1.92  1.00  0.54  3.9

All surprisal = [1.062]
Word completed by certainly len remaining = 0
['2', '15.682', '9.835', '4.328', '5.507', '1.474', '2.09', '0.88', '0.82', '2.8']
Word = certainly
Row = pfix:2      certainly  15.682  9.835 4.328  5.507  1.474  2.09  0.88  0.82  2.8

All surprisal = [4.328]
Word completed by did len remaining = 0
['3', '19.269', '3.587', '1.055', '2.531', '0.863', '2.00', '1.24', '1.24', '3.0']
Word = did
Row = pfix:3            did  19.269  3.587 1.055  2.531  0.863  2.00  1.24  1.24  3.0

All surprisal = [1.055]
Word completed by talk len remaining = 0
['4', '30.182', '10.913', '3.474', '7.438', '1.838', '3.22', '0.99', '0.70', '2.2']
Word = talk
Row = pfix:4           talk  30.182 10.913 3.474  7.438  1.838  

...

she snapped .
Word completed by she len remaining = 0
['1', '6.054', '6.054', '2.039', '4.016', '0.018', '2.00', '1.00', '1.00', '4.0']
Word = she
Row = pfix:1            she   6.054  6.054 2.039  4.016  0.018  2.00  1.00  1.00  4.0

All surprisal = [2.039]
Skipped snapped len remaining = 1
['2', '16.973', '10.919', '0.969', '9.949', '0.381', '2.00', '1.00', '0.88', '3.0']
Word completed by . len remaining = 0
['3', '21.289', '4.316', '4.314', '0.001', '0.688', '1.00', '0.66', '0.66', '2.0']
Word = snapped.
Row = pfix:3              .  21.289  4.316 4.314  0.001  0.688  1.00  0.66  0.66  2.0

All surprisal = [0.969, 4.314]

" Didn ' t you see what it was standing on ?  "
Skipped " len remaining = 6
Skipped Didn len remaining = 2
['1', '5.847', '5.847', '1.062', '4.785', '1.560', '1.92', '1.00', '0.54', '3.9']
Skipped ' len remaining = 1
['2', '12.671', '6.823', '3.152', '3.671', '1.178', '1.96', '1.51', '1.01', '1.0']
Word completed by t len remaining = 0
['3', '22.298', '9.628', '0.39

" No ,  not the floor .
Skipped " len remaining = 3
Skipped No len remaining = 1
['1', '6.315', '6.315', '0.870', '5.444', '0.884', '1.88', '1.00', '0.78', '3.9']
Word completed by , len remaining = 0
['2', '10.046', '3.731', '3.731', '0.000', '1.525', '1.03', '0.08', '0.00', '2.0']
Word = "No,
Row = pfix:2              ,  10.046  3.731 3.731  0.000  1.525  1.03  0.08  0.00  2.0

All surprisal = [0.87, 3.731]
Word completed by not len remaining = 0
['3', '16.891', '6.845', '2.755', '4.090', '2.399', '1.72', '0.42', '0.00', '1.6']
Word = not
Row = pfix:3            not  16.891  6.845 2.755  4.090  2.399  1.72  0.42  0.00  1.6

All surprisal = [2.755]
Word completed by the len remaining = 0
['4', '20.220', '3.329', '1.997', '1.332', '2.131', '2.28', '1.53', '1.49', '1.9']
Word = the
Row = pfix:4            the  20.220  3.329 1.997  1.332  2.131  2.28  1.53  1.49  1.9

All surprisal = [1.997]
Skipped floor len remaining = 1
['5', '28.004', '7.784', '0.755', '7.029', '2.174', '2.32', '1.01

Now ,  if you don ' t mind ,  I ' m going to bed .  "
Skipped Now len remaining = 1
['1', '5.672', '5.672', '2.939', '2.733', '0.307', '2.00', '1.00', '0.94', '4.0']
Word completed by , len remaining = 0
['2', '6.361', '0.689', '0.689', '0.000', '0.141', '1.00', '1.03', '1.03', '2.0']
Word = Now,
Row = pfix:2              ,   6.361  0.689 0.689  0.000  0.141  1.00  1.03  1.03  2.0

All surprisal = [2.939, 0.689]
Word completed by if len remaining = 0
['3', '11.935', '5.575', '1.620', '3.955', '0.294', '2.06', '1.02', '0.96', '2.1']
Word = if
Row = pfix:3             if  11.935  5.575 1.620  3.955  0.294  2.06  1.02  0.96  2.1

All surprisal = [1.62]
Word completed by you len remaining = 0
['4', '15.208', '3.272', '1.739', '1.533', '0.301', '4.06', '1.00', '1.00', '3.0']
Word = you
Row = pfix:4            you  15.208  3.272 1.739  1.533  0.301  4.06  1.00  1.00  3.0

All surprisal = [1.739]
Skipped don len remaining = 2
['5', '30.323', '15.115', '4.297', '10.818', '0.261', '4.05', '1.01

But Hermione had given Harry something else to think about as he climbed back into bed .
Word completed by But len remaining = 0
['1', '3.136', '3.136', '2.832', '0.305', '0.121', '1.00', '1.00', '0.98', '3.0']
Word = But
Row = pfix:1            But   3.136  3.136 2.832  0.305  0.121  1.00  1.00  0.98  3.0

All surprisal = [2.832]
Word completed by Hermione len remaining = 0
['2', '7.674', '4.538', '1.188', '3.350', '0.435', '2.05', '1.02', '0.93', '2.0']
Word = Hermione
Row = pfix:2       Hermione   7.674  4.538 1.188  3.350  0.435  2.05  1.02  0.93  2.0

All surprisal = [1.188]
Word completed by had len remaining = 0
['3', '13.529', '5.855', '2.584', '3.271', '0.527', '2.12', '0.94', '0.94', '3.0']
Word = had
Row = pfix:3            had  13.529  5.855 2.584  3.271  0.527  2.12  0.94  0.94  3.0

All surprisal = [2.584]
Word completed by given len remaining = 0
['4', '20.132', '6.603', '1.269', '5.334', '0.341', '3.04', '1.09', '1.09', '2.0']
Word = given
Row = pfix:4          given  2

It looked as though Harry had found out where the grubby little package from vault seven hundred and thirteen was .
Word completed by It len remaining = 0
['1', '3.584', '3.584', '2.389', '1.194', '0.007', '2.00', '1.00', '1.00', '4.0']
Word = It
Row = pfix:1             It   3.584  3.584 2.389  1.194  0.007  2.00  1.00  1.00  4.0

All surprisal = [2.389]
Word completed by looked len remaining = 0
['2', '10.743', '7.160', '1.019', '6.141', '0.020', '2.00', '1.00', '1.00', '3.0']
Word = looked
Row = pfix:2         looked  10.743  7.160 1.019  6.141  0.020  2.00  1.00  1.00  3.0

All surprisal = [1.019]
Word completed by as len remaining = 0
['3', '14.765', '4.022', '0.718', '3.304', '1.261', '3.01', '1.00', '0.56', '2.0']
Word = as
Row = pfix:3             as  14.765  4.022 0.718  3.304  1.261  3.01  1.00  0.56  2.0

All surprisal = [0.718]
Word completed by though len remaining = 0
['4', '22.887', '8.121', '2.415', '5.706', '1.356', '3.95', '0.93', '0.00', '2.0']
Word = though
Row = pf

In [5]:
len(all_surprisal)

5176

In [6]:
mean_surprisal = []
for i in range(len(all_surprisal)):
    print(hp_text[i])
    print(all_surprisal[i])
    if len(all_surprisal[i]) != 0:
        mean_surprisal.append(np.mean(all_surprisal[i]))
    else:
        mean_surprisal.append(0)
    print(mean_surprisal[-1])

np.save(os.path.join(save_dir,"syntactic_surprisal"), mean_surprisal)

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.
Harry
[1.6]
1.6
had
[4.133]
4.133
never
[2.707]
2.707
believed
[0.88]
0.88
he
[4.313]
4.313
would
[1.869]
1.869
meet
[0.178]
0.178
a
[1.501]
1.501
boy
[0.79]
0.79
he
[5.521]
5.521
hated
[5.37]
5.37
more
[5.158]
5.158
than
[0.249]
0.249
Dudley,
[6.571, 2.389]
4.4799999999999995
but
[2.407]
2.407
that
[1.522]
1.522
was
[2.122]
2.122
before
[1.424]
1.424
he
[2.609]
2.609
met
[1.259]
1.259
Draco
[0.873]
0.873
Malfoy.
[0.896, 2.233]
1.5645
Still,
[2.895, 0.163]
1.529
first-year
[1.122, 5.171, 3.949]
3.414
Gryffindors
[0.876]
0.876
only
[3.552]
3.552
had
[1.416]
1.416
Potions
[1.842]
1.842
with
[1.852]
1.852
the
[0.993]
0.993
Slytherins,
[0.354, 2.485]
1.4195
so
[2.248]
2.248
they
[1.861]
1.861
didn't
[2.412, 7.398, 0.668]
3.492666666666666
have
[2.369]
2.369
to
[2.975]
2.975
put
[0.124]
0.124
up
[1.307]
1.307
with
[2.597]
2.597
Malfoy
[1.207]
1.20

1.16
on
[1.86]
1.86
the
[1.053]
1.053
grass
[0.783]
0.783
in
[1.525]
1.525
a
[1.124]
1.124
heap.
[0.769, 2.231]
1.5
His
[4.641]
4.641
broomstick
[0.041]
0.041
was
[1.835]
1.835
still
[2.052]
2.052
rising
[1.973]
1.973
higher
[4.132]
4.132
and
[3.099]
3.099
higher,
[0.802, 3.043]
1.9225
and
[1.682]
1.682
started
[2.399]
2.399
to
[2.774]
2.774
drift
[0.459]
0.459
lazily
[3.296]
3.296
toward
[1.424]
1.424
the
[0.981]
0.981
forbidden
[5.168]
5.168
forest
[0.525]
0.525
and
[3.333]
3.333
out
[2.49]
2.49
of
[2.782]
2.782
sight.
[2.161, 1.608]
1.8845
+
[]
0
Madam
[1.062]
1.062
Hooch
[0.756]
0.756
was
[2.068]
2.068
bending
[3.036]
3.036
over
[1.265]
1.265
Neville,
[1.264, 2.655]
1.9594999999999998
her
[2.869]
2.869
face
[0.45]
0.45
as
[1.637]
1.637
white
[2.501]
2.501
as
[4.057]
4.057
his.
[3.444, -1.529]
0.9575
"Broken
[3.037]
3.037
wrist,"
[1.001, 2.393]
1.6969999999999998
Harry
[2.893]
2.893
heard
[0.403]
0.403
her
[2.828]
2.828
mutter.
[0.498, 2.53]
1.5139999999999998
"Come
[4.612]
4.612
on

seen
[5.144]
5.144
a
[2.49]
2.49
game
[0.634]
0.634
of
[1.098]
1.098
Quidditch,
[0.788, 2.252]
1.52
Potter?"
[1.685, 5.848]
3.7664999999999997
he
[2.389]
2.389
asked
[0.997]
0.997
excitedly.
[1.757, 3.391]
2.574
+
[]
0
"Wood's
[1.595, 2.751, 5.182]
3.176
captain
[3.435]
3.435
of
[1.365]
1.365
the
[1.138]
1.138
Gryffindor
[0.124]
0.124
team,"
[0.691, 1.991]
1.341
Professor
[2.337]
2.337
McGonagall
[1.196]
1.196
explained.
[2.676, 0.27]
1.473
+
[]
0
"He's
[2.389, 6.3, 1.962]
3.550333333333333
just
[1.692]
1.692
the
[2.048]
2.048
build
[4.626]
4.626
for
[2.098]
2.098
a
[0.937]
0.937
Seeker,
[0.173, 3.189]
1.681
too,"
[3.598, 2.483]
3.0404999999999998
said
[4.407]
4.407
Wood,
[0.63, 6.621]
3.6255
now
[3.597]
3.597
walking
[4.015]
4.015
around
[2.189]
2.189
Harry
[2.225]
2.225
and
[3.246]
3.246
staring
[3.183]
3.183
at
[1.865]
1.865
him.
[4.007, 1.609]
2.808
"Light
[1.595]
1.595
--
[3.62, 11.278]
7.449
speedy
[4.544]
4.544
--
[6.984, 5.343]
6.1635
we'll
[2.352, 5.587, 1.06]
2.99966666666666

[6.829]
6.829
up
[1.435]
1.435
on
[1.019]
1.019
the
[0.84]
0.84
floor,
[0.712, 2.351]
1.5314999999999999
fast
[2.702]
2.702
asleep,
[3.046, 2.056]
2.551
but
[1.415]
1.415
jerked
[4.898]
4.898
suddenly
[2.851]
2.851
awake
[2.009]
2.009
as
[2.777]
2.777
they
[3.09]
3.09
crept
[1.275]
1.275
nearer.
[2.104, 11.045]
6.5745000000000005
+
[]
0
"Thank
[5.633]
5.633
goodness
[2.549]
2.549
you
[4.568]
4.568
found
[1.18]
1.18
me!
[2.945, 3.234]
3.0895
I've
[1.152, 6.136, 2.874]
3.3873333333333338
been
[4.975]
4.975
out
[0.901]
0.901
here
[3.827]
3.827
for
[2.007]
2.007
hours,
[2.926, 1.617]
2.2715
I
[3.617]
3.617
couldn't
[2.219, 6.865, 0.943]
3.342333333333333
remember
[4.272]
4.272
the
[2.137]
2.137
new
[1.72]
1.72
password
[0.159]
0.159
to
[2.867]
2.867
get
[0.833]
0.833
in
[1.729]
1.729
to
[4.218]
4.218
bed."
[2.63, 1.551]
2.0905
+
[]
0
"Keep
[5.608]
5.608
your
[2.552]
2.552
voice
[0.398]
0.398
down,
[1.241, 1.342]
1.2915
Neville.
[0.97, 9.797]
5.383500000000001
The
[0.878]
0.878
password's
[

1.635
them,
[1.036, 1.991]
1.5135
and
[1.434]
1.434
Harry
[2.472]
2.472
knew
[2.447]
2.447
that
[1.483]
1.483
the
[1.36]
1.36
only
[1.904]
1.904
reason
[0.35]
0.35
they
[6.33]
6.33
weren't
[2.202, 8.127, 2.935]
4.421333333333334
already
[4.618]
4.618
dead
[3.132]
3.132
was
[6.417]
6.417
that
[1.523]
1.523
their
[3.432]
3.432
sudden
[2.121]
2.121
appearance
[0.439]
0.439
had
[1.917]
1.917
taken
[0.946]
0.946
it
[4.721]
4.721
by
[1.512]
1.512
surprise,
[1.646, 1.93]
1.7879999999999998
but
[1.682]
1.682
it
[2.428]
2.428
was
[1.217]
1.217
quickly
[1.86]
1.86
getting
[3.056]
3.056
over
[1.337]
1.337
that,
[1.262, 6.111]
3.6864999999999997
there
[3.538]
3.538
was
[1.687]
1.687
no
[1.642]
1.642
mistaking
[0.255]
0.255
what
[5.362]
5.362
those
[3.746]
3.746
thunderous
[1.747]
1.747
growls
[4.298]
4.298
meant.
[1.864, 5.41]
3.637
+
[]
0
Harry
[1.6]
1.6
groped
[0.111]
0.111
for
[2.063]
2.063
the
[1.026]
1.026
doorknob
[0.089]
0.089
--
[5.251, 6.265]
5.758
between
[2.305]
2.305
Filch
[1.028]
1.02